# データサイエンス100本ノック（構造化データ加工編） - Python

## はじめに
- 初めに以下のセルを実行してください
- 必要なライブラリのインポートとデータベース（PostgreSQL）からのデータ読み込みを行います
- pandas等、利用が想定されるライブラリは以下セルでインポートしています
- その他利用したいライブラリがあれば適宜インストールしてください（"!pip install ライブラリ名"でインストールも可能）
- 処理は複数回に分けても構いません
- 名前、住所等はダミーデータであり、実在するものではありません

In [1]:
import os
import pandas as pd
import numpy as np
from datetime import datetime, date
from dateutil.relativedelta import relativedelta
import math
import psycopg2
from sqlalchemy import create_engine
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from imblearn.under_sampling import RandomUnderSampler

os.environ.pop('PG_PORT', None) # postgresから読むとコケることがあるため、CSVを読む分岐に入らせる(s.f追加)

if 'PG_PORT' in os.environ:
    pgconfig = {
        'host': 'db',
        'port': os.environ['PG_PORT'],
        'database': os.environ['PG_DATABASE'],
        'user': os.environ['PG_USER'],
        'password': os.environ['PG_PASSWORD'],
    }


    # pd.read_sql用のコネクタ
    conn = psycopg2.connect(**pgconfig)

    df_customer = pd.read_sql(sql='select * from customer', con=conn)
    df_category = pd.read_sql(sql='select * from category', con=conn)
    df_product = pd.read_sql(sql='select * from product', con=conn)
    df_receipt = pd.read_sql(sql='select * from receipt', con=conn)
    df_store = pd.read_sql(sql='select * from store', con=conn)
    df_geocode = pd.read_sql(sql='select * from geocode', con=conn)

else:
    if not os.path.exists('data/'):
        !git clone https://github.com/The-Japan-DataScientist-Society/100knocks-preprocess
        os.chdir('100knocks-preprocess/docker/work/')

    dtype = {
        'customer_id': str,
        'gender_cd': str,
        'postal_cd': str,
        'application_store_cd': str,
        'status_cd': str,
        'category_major_cd': str,
        'category_medium_cd': str,
        'category_small_cd': str,
        'product_cd': str,
        'store_cd': str,
        'prefecture_cd': str,
        'tel_no': str,
        'postal_cd': str,
        'street': str
    }

    df_customer = pd.read_csv("data/customer.csv", dtype=dtype)
    df_category = pd.read_csv("data/category.csv", dtype=dtype)
    df_product = pd.read_csv("data/product.csv", dtype=dtype)
    df_receipt = pd.read_csv("data/receipt.csv", dtype=dtype)
    df_store = pd.read_csv("data/store.csv", dtype=dtype)
    df_geocode = pd.read_csv("data/geocode.csv", dtype=dtype)


# 演習問題

---
> P-001: レシート明細データ（df_receipt）から全項目の先頭10件を表示し、どのようなデータを保有しているか目視で確認せよ。

In [6]:
df_receipt.head(10)

sales_ymd  sales_epoch store_cd  receipt_no  receipt_sub_no  \
0   20181103   1541203200   S14006         112               1   
1   20181118   1542499200   S13008        1132               2   
2   20170712   1499817600   S14028        1102               1   
3   20190205   1549324800   S14042        1132               1   
4   20180821   1534809600   S14025        1102               2   
5   20190605   1559692800   S13003        1112               1   
6   20181205   1543968000   S14024        1102               2   
7   20190922   1569110400   S14040        1102               1   
8   20170504   1493856000   S13020        1112               2   
9   20191010   1570665600   S14027        1102               1   

      customer_id  product_cd  quantity  amount  
0  CS006214000001  P070305012         1     158  
1  CS008415000097  P070701017         1      81  
2  CS028414000014  P060101005         1     170  
3  ZZ000000000000  P050301001         1      25  
4  CS025415000050  P060102007         1      90  
5  CS003515000195  P050102002         1     138  
6  CS024514000042  P080101005         1      30  
7  CS040415000178  P070501004         1     128  
8  ZZ000000000000  P071302010         1     770  
9  CS027514000015  P071101003         1     680

---
> P-002: レシート明細データ（df_receipt）から売上年月日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示せよ。

In [4]:
df_receipt[['sales_ymd', 'customer_id', 'product_cd', 'amount']].head(10)

sales_ymd     customer_id  product_cd  amount
0   20181103  CS006214000001  P070305012     158
1   20181118  CS008415000097  P070701017      81
2   20170712  CS028414000014  P060101005     170
3   20190205  ZZ000000000000  P050301001      25
4   20180821  CS025415000050  P060102007      90
5   20190605  CS003515000195  P050102002     138
6   20181205  CS024514000042  P080101005      30
7   20190922  CS040415000178  P070501004     128
8   20170504  ZZ000000000000  P071302010     770
9   20191010  CS027514000015  P071101003     680

---
> P-003: レシート明細データ（df_receipt）から売上年月日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示せよ。ただし、sales_ymdsales_dateに項目名を変更しながら抽出すること。

In [2]:
df_receipt[['sales_ymd', 'customer_id', 'product_cd', 'amount']].rename(columns={'sales_ymd': 'sales_date'}).head(10)

sales_date     customer_id  product_cd  amount
0    20181103  CS006214000001  P070305012     158
1    20181118  CS008415000097  P070701017      81
2    20170712  CS028414000014  P060101005     170
3    20190205  ZZ000000000000  P050301001      25
4    20180821  CS025415000050  P060102007      90
5    20190605  CS003515000195  P050102002     138
6    20181205  CS024514000042  P080101005      30
7    20190922  CS040415000178  P070501004     128
8    20170504  ZZ000000000000  P071302010     770
9    20191010  CS027514000015  P071101003     680

---
> P-004: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"

In [3]:
df_receipt[['sales_ymd', 'customer_id', 'product_cd', 'amount']][df_receipt.customer_id == 'CS018205000001']

sales_ymd     customer_id  product_cd  amount
36      20180911  CS018205000001  P071401012    2200
9843    20180414  CS018205000001  P060104007     600
21110   20170614  CS018205000001  P050206001     990
27673   20170614  CS018205000001  P060702015     108
27840   20190216  CS018205000001  P071005024     102
28757   20180414  CS018205000001  P071101002     278
39256   20190226  CS018205000001  P070902035     168
58121   20190924  CS018205000001  P060805001     495
68117   20190226  CS018205000001  P071401020    2200
72254   20180911  CS018205000001  P071401005    1100
88508   20190216  CS018205000001  P040101002     218
91525   20190924  CS018205000001  P091503001     280

---
> P-005: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上

In [4]:
df_receipt[['sales_ymd', 'customer_id', 'product_cd', 'amount']][(df_receipt.customer_id == 'CS018205000001') & (df_receipt.amount >=1000)]

sales_ymd     customer_id  product_cd  amount
36      20180911  CS018205000001  P071401012    2200
68117   20190226  CS018205000001  P071401020    2200
72254   20180911  CS018205000001  P071401005    1100

In [5]:
df_receipt[['sales_ymd', 'customer_id', 'product_cd', 'amount']].query('customer_id == "CS018205000001" and amount >= 1000')

sales_ymd     customer_id  product_cd  amount
36      20180911  CS018205000001  P071401012    2200
68117   20190226  CS018205000001  P071401020    2200
72254   20180911  CS018205000001  P071401005    1100

---
> P-006: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上数量（quantity）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上または売上数量（quantity）が5以上

In [8]:
df_receipt[['sales_ymd', 'customer_id', 'product_cd', 'quantity', 'amount']].query('customer_id == "CS018205000001" and (amount >= 1000 or quantity >= 5)')

sales_ymd     customer_id  product_cd  quantity  amount
36      20180911  CS018205000001  P071401012         1    2200
9843    20180414  CS018205000001  P060104007         6     600
21110   20170614  CS018205000001  P050206001         5     990
68117   20190226  CS018205000001  P071401020         1    2200
72254   20180911  CS018205000001  P071401005         1    1100

---
> P-007: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上2,000以下

In [10]:
df_receipt[['sales_ymd', 'customer_id', 'product_cd', 'amount']].query('customer_id == "CS018205000001" and amount >= 1000 and amount <= 2000')

sales_ymd     customer_id  product_cd  amount
72254   20180911  CS018205000001  P071401005    1100

---
> P-008: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 商品コード（product_cd）が"P071401019"以外

In [11]:
df_receipt[['sales_ymd', 'customer_id', 'product_cd', 'amount']].query('customer_id == "CS018205000001" and product_cd != "P071401019"')

sales_ymd     customer_id  product_cd  amount
36      20180911  CS018205000001  P071401012    2200
9843    20180414  CS018205000001  P060104007     600
21110   20170614  CS018205000001  P050206001     990
27673   20170614  CS018205000001  P060702015     108
27840   20190216  CS018205000001  P071005024     102
28757   20180414  CS018205000001  P071101002     278
39256   20190226  CS018205000001  P070902035     168
58121   20190924  CS018205000001  P060805001     495
68117   20190226  CS018205000001  P071401020    2200
72254   20180911  CS018205000001  P071401005    1100
88508   20190216  CS018205000001  P040101002     218
91525   20190924  CS018205000001  P091503001     280

---
> P-009: 以下の処理において、出力結果を変えずにORをANDに書き換えよ。
> 
> `df_store.query('not(prefecture_cd == "13" | floor_area > 900)')`

In [12]:
df_store.query('prefecture_cd != "13" and floor_area <= 900')

store_cd store_name prefecture_cd prefecture            address  \
18   S14046       北山田店            14       神奈川県   神奈川県横浜市都筑区北山田一丁目   
20   S14011      日吉本町店            14       神奈川県  神奈川県横浜市港北区日吉本町四丁目   
38   S12013       習志野店            12        千葉県       千葉県習志野市芝園一丁目   

                      address_kana        tel_no  longitude  latitude  \
18      カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ  045-123-4049   139.5916  35.56189   
20  カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ  045-123-4033   139.6316  35.54655   
38             チバケンナラシノシシバゾノイッチョウメ  047-123-4002   140.0220  35.66122   

    floor_area  
18       831.0  
20       890.0  
38       808.0

---
> P-010: 店舗データ（df_store）から、店舗コード（store_cd）が"S14"で始まるものだけ全項目抽出し、10件表示せよ。

In [14]:
df_store[df_store.store_cd.str.startswith('S14')].head(10)

store_cd store_name prefecture_cd prefecture              address  \
2    S14010        菊名店            14       神奈川県      神奈川県横浜市港北区菊名一丁目   
3    S14033       阿久和店            14       神奈川県    神奈川県横浜市瀬谷区阿久和西一丁目   
4    S14036     相模原中央店            14       神奈川県        神奈川県相模原市中央二丁目   
7    S14040       長津田店            14       神奈川県  神奈川県横浜市緑区長津田みなみ台五丁目   
9    S14050      阿久和西店            14       神奈川県    神奈川県横浜市瀬谷区阿久和西一丁目   
12   S14028       二ツ橋店            14       神奈川県       神奈川県横浜市瀬谷区二ツ橋町   
16   S14012      本牧和田店            14       神奈川県        神奈川県横浜市中区本牧和田   
18   S14046       北山田店            14       神奈川県     神奈川県横浜市都筑区北山田一丁目   
19   S14022        逗子店            14       神奈川県         神奈川県逗子市逗子一丁目   
20   S14011      日吉本町店            14       神奈川県    神奈川県横浜市港北区日吉本町四丁目   

                      address_kana        tel_no  longitude  latitude  \
2        カナガワケンヨコハマシコウホククキクナイッチョウメ  045-123-4032   139.6326  35.50049   
3        カナガワケンヨコハマシセヤクアクワニシイッチョウメ  045-123-4043   139.4961  35.45918   
4           カナガワケンサガミハラシチュウオウニチョウメ  042-123-4045   139.3716  35.57327   
7    カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ  045-123-4046   139.4994  35.52398   
9        カナガワケンヨコハマシセヤクアクワニシイッチョウメ  045-123-4053   139.4961  35.45918   
12          カナガワケンヨコハマシセヤクフタツバシチョウ  045-123-4042   139.4963  35.46304   
16            カナガワケンヨコハマシナカクホンモクワダ  045-123-4034   139.6582  35.42156   
18      カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ  045-123-4049   139.5916  35.56189   
19               カナガワケンズシシズシイッチョウメ  046-123-4036   139.5789  35.29642   
20  カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ  045-123-4033   139.6316  35.54655   

    floor_area  
2       1732.0  
3       1495.0  
4       1679.0  
7       1548.0  
9       1830.0  
12      1574.0  
16      1341.0  
18       831.0  
19      1838.0  
20       890.0

In [15]:
df_store.query('store_cd.str.startswith("S14")').head(10)

store_cd store_name prefecture_cd prefecture              address  \
2    S14010        菊名店            14       神奈川県      神奈川県横浜市港北区菊名一丁目   
3    S14033       阿久和店            14       神奈川県    神奈川県横浜市瀬谷区阿久和西一丁目   
4    S14036     相模原中央店            14       神奈川県        神奈川県相模原市中央二丁目   
7    S14040       長津田店            14       神奈川県  神奈川県横浜市緑区長津田みなみ台五丁目   
9    S14050      阿久和西店            14       神奈川県    神奈川県横浜市瀬谷区阿久和西一丁目   
12   S14028       二ツ橋店            14       神奈川県       神奈川県横浜市瀬谷区二ツ橋町   
16   S14012      本牧和田店            14       神奈川県        神奈川県横浜市中区本牧和田   
18   S14046       北山田店            14       神奈川県     神奈川県横浜市都筑区北山田一丁目   
19   S14022        逗子店            14       神奈川県         神奈川県逗子市逗子一丁目   
20   S14011      日吉本町店            14       神奈川県    神奈川県横浜市港北区日吉本町四丁目   

                      address_kana        tel_no  longitude  latitude  \
2        カナガワケンヨコハマシコウホククキクナイッチョウメ  045-123-4032   139.6326  35.50049   
3        カナガワケンヨコハマシセヤクアクワニシイッチョウメ  045-123-4043   139.4961  35.45918   
4           カナガワケンサガミハラシチュウオウニチョウメ  042-123-4045   139.3716  35.57327   
7    カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ  045-123-4046   139.4994  35.52398   
9        カナガワケンヨコハマシセヤクアクワニシイッチョウメ  045-123-4053   139.4961  35.45918   
12          カナガワケンヨコハマシセヤクフタツバシチョウ  045-123-4042   139.4963  35.46304   
16            カナガワケンヨコハマシナカクホンモクワダ  045-123-4034   139.6582  35.42156   
18      カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ  045-123-4049   139.5916  35.56189   
19               カナガワケンズシシズシイッチョウメ  046-123-4036   139.5789  35.29642   
20  カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ  045-123-4033   139.6316  35.54655   

    floor_area  
2       1732.0  
3       1495.0  
4       1679.0  
7       1548.0  
9       1830.0  
12      1574.0  
16      1341.0  
18       831.0  
19      1838.0  
20       890.0

---
> P-011: 顧客データ（df_customer）から顧客ID（customer_id）の末尾が1のものだけ全項目抽出し、10件表示せよ。

In [16]:
df_customer.query('customer_id.str.endswith("1")')

customer_id customer_name gender_cd gender   birth_day  age  \
1      CS037613000071         六角 雅彦         9     不明  1952-04-01   66   
3      CS028811000001        堀井 かおり         1     女性  1933-03-27   86   
14     CS040412000191         川井 郁恵         1     女性  1977-01-05   42   
31     CS028314000011        小菅 あおい         1     女性  1983-11-26   35   
56     CS039212000051        藤島 恵梨香         1     女性  1997-02-03   22   
...               ...           ...       ...    ...         ...  ...   
21927  CS020614000011         河内 一徳         9     不明  1954-05-22   64   
21942  CS007415000061        緒方 貴美子         1     女性  1971-06-08   47   
21944  CS021615000081          小柳 遥         1     女性  1950-05-21   68   
21947  CS005313000401          木下 優         1     女性  1982-06-24   36   
21949  CS018313000061         竹内 恭子         1     女性  1979-01-04   40   

      postal_cd                  address application_store_cd  \
1      136-0076       東京都江東区南砂**********               S13037   
3      245-0016   神奈川県横浜市泉区和泉町**********               S14028   
14     226-0021  神奈川県横浜市緑区北八朔町**********               S14040   
31     246-0038   神奈川県横浜市瀬谷区宮沢**********               S14028   
56     166-0001     東京都杉並区阿佐谷北**********               S13039   
...         ...                      ...                  ...   
21927  114-0032       東京都北区中十条**********               S13020   
21942  276-0022     千葉県八千代市上高野**********               S12007   
21944  259-1133    神奈川県伊勢原市東大竹**********               S14021   
21947  167-0022      東京都杉並区下井草**********               S13005   
21949  203-0051     東京都東久留米市小山**********               S13018   

      application_date     status_cd  
1             20150414  0-00000000-0  
3             20160115  0-00000000-0  
14            20151101  1-20091025-4  
31            20151123  1-20080426-5  
56            20171121  1-20100215-4  
...                ...           ...  
21927         20151212  8-20100802-2  
21942         20141130  0-00000000-0  
21944         20150627  D-20100710-E  
21947         20161221  0-00000000-0  
21949         20150508  0-00000000-0  

[2369 rows x 11 columns]

---
> P-012: 店舗データ（df_store）から、住所 (address) に"横浜市"が含まれるものだけ全項目表示せよ。

In [17]:
df_store.query('address.str.contains("横浜市")')

store_cd store_name prefecture_cd prefecture              address  \
2    S14010        菊名店            14       神奈川県      神奈川県横浜市港北区菊名一丁目   
3    S14033       阿久和店            14       神奈川県    神奈川県横浜市瀬谷区阿久和西一丁目   
7    S14040       長津田店            14       神奈川県  神奈川県横浜市緑区長津田みなみ台五丁目   
9    S14050      阿久和西店            14       神奈川県    神奈川県横浜市瀬谷区阿久和西一丁目   
12   S14028       二ツ橋店            14       神奈川県       神奈川県横浜市瀬谷区二ツ橋町   
16   S14012      本牧和田店            14       神奈川県        神奈川県横浜市中区本牧和田   
18   S14046       北山田店            14       神奈川県     神奈川県横浜市都筑区北山田一丁目   
20   S14011      日吉本町店            14       神奈川県    神奈川県横浜市港北区日吉本町四丁目   
26   S14048      中川中央店            14       神奈川県    神奈川県横浜市都筑区中川中央二丁目   
40   S14042       新山下店            14       神奈川県      神奈川県横浜市中区新山下二丁目   
52   S14006       葛が谷店            14       神奈川県        神奈川県横浜市都筑区葛が谷   

                      address_kana        tel_no  longitude  latitude  \
2        カナガワケンヨコハマシコウホククキクナイッチョウメ  045-123-4032   139.6326  35.50049   
3        カナガワケンヨコハマシセヤクアクワニシイッチョウメ  045-123-4043   139.4961  35.45918   
7    カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ  045-123-4046   139.4994  35.52398   
9        カナガワケンヨコハマシセヤクアクワニシイッチョウメ  045-123-4053   139.4961  35.45918   
12          カナガワケンヨコハマシセヤクフタツバシチョウ  045-123-4042   139.4963  35.46304   
16            カナガワケンヨコハマシナカクホンモクワダ  045-123-4034   139.6582  35.42156   
18      カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ  045-123-4049   139.5916  35.56189   
20  カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ  045-123-4033   139.6316  35.54655   
26   カナガワケンヨコハマシツヅキクナカガワチュウオウニチョウメ  045-123-4051   139.5758  35.54912   
40       カナガワケンヨコハマシナカクシンヤマシタニチョウメ  045-123-4047   139.6593  35.43894   
52             カナガワケンヨコハマシツヅキククズガヤ  045-123-4031   139.5633  35.53573   

    floor_area  
2       1732.0  
3       1495.0  
7       1548.0  
9       1830.0  
12      1574.0  
16      1341.0  
18       831.0  
20       890.0  
26      1657.0  
40      1044.0  
52      1886.0

---
> P-013: 顧客データ（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まるデータを全項目抽出し、10件表示せよ。

In [22]:
df_customer.query('status_cd.str.contains(r"^[A-Z]")').head(10)

customer_id customer_name gender_cd gender   birth_day  age postal_cd  \
2   CS031415000172       宇多田 貴美子         1     女性  1976-10-04   42  151-0053   
6   CS015414000103         奥野 陽子         1     女性  1977-08-09   41  136-0073   
12  CS011215000048         芦田 沙耶         1     女性  1992-02-01   27  223-0062   
15  CS029415000023         梅田 里穂         1     女性  1976-01-17   43  279-0043   
21  CS035415000029         寺沢 真希         9     不明  1977-09-27   41  158-0096   
32  CS031415000106        宇野 由美子         1     女性  1970-02-26   49  151-0053   
33  CS029215000025         石倉 美帆         1     女性  1993-09-28   25  279-0022   
40  CS033605000005         猪股 雄太         0     男性  1955-12-05   63  246-0031   
44  CS033415000229        板垣 菜々美         1     女性  1977-11-07   41  246-0021   
53  CS008415000145         黒谷 麻緒         1     女性  1977-06-27   41  157-0067   

                     address application_store_cd application_date  \
2        東京都渋谷区代々木**********               S13031         20150529   
6         東京都江東区北砂**********               S13015         20150722   
12  神奈川県横浜市港北区日吉本町**********               S14011         20150228   
15       千葉県浦安市富士見**********               S12029         20150610   
21      東京都世田谷区玉川台**********               S13035         20141220   
32       東京都渋谷区代々木**********               S13031         20150201   
33        千葉県浦安市今川**********               S12029         20150708   
40    神奈川県横浜市瀬谷区瀬谷**********               S14033         20150425   
44  神奈川県横浜市瀬谷区二ツ橋町**********               S14033         20150712   
53      東京都世田谷区喜多見**********               S13008         20150829   

       status_cd  
2   D-20100325-C  
6   B-20100609-B  
12  C-20100421-9  
15  D-20100918-E  
21  F-20101029-F  
32  F-20100511-E  
33  B-20100820-C  
40  F-20100917-E  
44  F-20100326-E  
53  F-20100622-F

---
> P-014: 顧客データ（df_customer）から、ステータスコード（status_cd）の末尾が数字の1〜9で終わるデータを全項目抽出し、10件表示せよ。

In [4]:
df_customer.query("status_cd.str.contains('[1-9]$')").head(10)

customer_id customer_name gender_cd gender   birth_day  age postal_cd  \
4   CS001215000145         田崎 美紀         1     女性  1995-03-29   24  144-0055   
9   CS033513000180          安斎 遥         1     女性  1962-07-11   56  241-0823   
12  CS011215000048         芦田 沙耶         1     女性  1992-02-01   27  223-0062   
14  CS040412000191         川井 郁恵         1     女性  1977-01-05   42  226-0021   
16  CS009315000023         皆川 文世         1     女性  1980-04-15   38  154-0012   
22  CS015315000033        福士 璃奈子         1     女性  1983-03-17   36  135-0043   
23  CS023513000066         神戸 そら         1     女性  1961-12-17   57  210-0005   
24  CS035513000134         市川 美帆         1     女性  1960-03-27   59  156-0053   
27  CS001515000263         高松 夏空         1     女性  1962-11-09   56  144-0051   
28  CS040314000027       鶴田 きみまろ         9     不明  1986-03-26   33  226-0027   

                     address application_store_cd application_date  \
4        東京都大田区仲六郷**********               S13001         20170605   
9     神奈川県横浜市旭区善部町**********               S14033         20150728   
12  神奈川県横浜市港北区日吉本町**********               S14011         20150228   
14   神奈川県横浜市緑区北八朔町**********               S14040         20151101   
16       東京都世田谷区駒沢**********               S13009         20150319   
22        東京都江東区塩浜**********               S13015         20141024   
23   神奈川県川崎市川崎区東田町**********               S14023         20150915   
24        東京都世田谷区桜**********               S13035         20150227   
27       東京都大田区西蒲田**********               S13001         20160812   
28    神奈川県横浜市緑区長津田**********               S14040         20150122   

       status_cd  
4   6-20090929-2  
9   6-20080506-5  
12  C-20100421-9  
14  1-20091025-4  
16  5-20080322-1  
22  4-20080219-3  
23  5-20100524-9  
24  8-20100711-9  
27  1-20100804-1  
28  2-20080426-4

---
> P-015: 顧客データ（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まり、末尾が数字の1〜9で終わるデータを全項目抽出し、10件表示せよ。

In [5]:
df_customer.query("status_cd.str.contains('^[A-Z].*[1-9]$')").head(10)

customer_id customer_name gender_cd gender   birth_day  age postal_cd  \
12   CS011215000048         芦田 沙耶         1     女性  1992-02-01   27  223-0062   
68   CS022513000105        島村 貴美子         1     女性  1962-03-12   57  249-0002   
71   CS001515000096         水野 陽子         9     不明  1960-11-29   58  144-0053   
122  CS013615000053         西脇 季衣         1     女性  1953-10-18   65  261-0026   
144  CS020412000161          小宮 薫         1     女性  1974-05-21   44  174-0042   
178  CS001215000097        竹中 あさみ         1     女性  1990-07-25   28  146-0095   
252  CS035212000007        内村 恵梨香         1     女性  1990-12-04   28  152-0023   
259  CS002515000386         野田 コウ         1     女性  1963-05-30   55  185-0013   
293  CS001615000372        稲垣 寿々花         1     女性  1956-10-29   62  144-0035   
297  CS032512000121         松井 知世         1     女性  1962-09-04   56  210-0011   

                      address application_store_cd application_date  \
12   神奈川県横浜市港北区日吉本町**********               S14011         20150228   
68       神奈川県逗子市山の根**********               S14022         20150320   
71       東京都大田区蒲田本町**********               S13001         20150614   
122    千葉県千葉市美浜区幕張西**********               S12013         20150128   
144       東京都板橋区東坂下**********               S13020         20150822   
178       東京都大田区多摩川**********               S13001         20170315   
252        東京都目黒区八雲**********               S13035         20151013   
259     東京都国分寺市西恋ケ窪**********               S13002         20160410   
293       東京都大田区南蒲田**********               S13001         20170403   
297   神奈川県川崎市川崎区富士見**********               S13032         20150727   

        status_cd  
12   C-20100421-9  
68   A-20091115-7  
71   A-20100724-7  
122  B-20100329-6  
144  B-20081021-3  
178  A-20100211-2  
252  B-20101018-6  
259  C-20100127-8  
293  A-20100104-1  
297  A-20100103-5

---
> P-016: 店舗データ（df_store）から、電話番号（tel_no）が3桁-3桁-4桁のデータを全項目表示せよ。

In [8]:
df_store.query("tel_no.str.match('\d{3}-\d{3}-\d{4}')")

store_cd store_name prefecture_cd prefecture              address  \
0    S12014       千草台店            12        千葉県      千葉県千葉市稲毛区千草台一丁目   
1    S13002       国分寺店            13        東京都         東京都国分寺市本多二丁目   
2    S14010        菊名店            14       神奈川県      神奈川県横浜市港北区菊名一丁目   
3    S14033       阿久和店            14       神奈川県    神奈川県横浜市瀬谷区阿久和西一丁目   
4    S14036     相模原中央店            14       神奈川県        神奈川県相模原市中央二丁目   
7    S14040       長津田店            14       神奈川県  神奈川県横浜市緑区長津田みなみ台五丁目   
9    S14050      阿久和西店            14       神奈川県    神奈川県横浜市瀬谷区阿久和西一丁目   
11   S13052        森野店            13        東京都          東京都町田市森野三丁目   
12   S14028       二ツ橋店            14       神奈川県       神奈川県横浜市瀬谷区二ツ橋町   
16   S14012      本牧和田店            14       神奈川県        神奈川県横浜市中区本牧和田   
18   S14046       北山田店            14       神奈川県     神奈川県横浜市都筑区北山田一丁目   
19   S14022        逗子店            14       神奈川県         神奈川県逗子市逗子一丁目   
20   S14011      日吉本町店            14       神奈川県    神奈川県横浜市港北区日吉本町四丁目   
21   S13016       小金井店            13        東京都         東京都小金井市本町一丁目   
22   S14034      川崎野川店            14       神奈川県         神奈川県川崎市宮前区野川   
26   S14048      中川中央店            14       神奈川県    神奈川県横浜市都筑区中川中央二丁目   
27   S12007        佐倉店            12        千葉県            千葉県佐倉市上志津   
28   S14026     辻堂西海岸店            14       神奈川県      神奈川県藤沢市辻堂西海岸二丁目   
29   S13041       八王子店            13        東京都            東京都八王子市大塚   
31   S14049      川崎大師店            14       神奈川県      神奈川県川崎市川崎区中瀬三丁目   
32   S14023        川崎店            14       神奈川県      神奈川県川崎市川崎区本町二丁目   
33   S13018        清瀬店            13        東京都          東京都清瀬市松山一丁目   
35   S14027       南藤沢店            14       神奈川県           神奈川県藤沢市南藤沢   
36   S14021       伊勢原店            14       神奈川県       神奈川県伊勢原市伊勢原四丁目   
37   S14047       相模原店            14       神奈川県       神奈川県相模原市千代田六丁目   
38   S12013       習志野店            12        千葉県         千葉県習志野市芝園一丁目   
40   S14042       新山下店            14       神奈川県      神奈川県横浜市中区新山下二丁目   
42   S12030        八幡店            12        千葉県          千葉県市川市八幡三丁目   
44   S14025        大和店            14       神奈川県           神奈川県大和市下和田   
45   S14045        厚木店            14       神奈川県         神奈川県厚木市中町二丁目   
47   S12029        東野店            12        千葉県          千葉県浦安市東野一丁目   
49   S12053        高洲店            12        千葉県          千葉県浦安市高洲五丁目   
51   S14024        三田店            14       神奈川県      神奈川県川崎市多摩区三田四丁目   
52   S14006       葛が谷店            14       神奈川県        神奈川県横浜市都筑区葛が谷   

                      address_kana        tel_no  longitude  latitude  \
0           チバケンチバシイナゲクチグサダイイッチョウメ  043-123-4003   140.1180  35.63559   
1             トウキョウトコクブンジシホンダニチョウメ  042-123-4008   139.4802  35.70566   
2        カナガワケンヨコハマシコウホククキクナイッチョウメ  045-123-4032   139.6326  35.50049   
3        カナガワケンヨコハマシセヤクアクワニシイッチョウメ  045-123-4043   139.4961  35.45918   
4           カナガワケンサガミハラシチュウオウニチョウメ  042-123-4045   139.3716  35.57327   
7    カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ  045-123-4046   139.4994  35.52398   
9        カナガワケンヨコハマシセヤクアクワニシイッチョウメ  045-123-4053   139.4961  35.45918   
11             トウキョウトマチダシモリノサンチョウメ  042-123-4030   139.4383  35.55293   
12          カナガワケンヨコハマシセヤクフタツバシチョウ  045-123-4042   139.4963  35.46304   
16            カナガワケンヨコハマシナカクホンモクワダ  045-123-4034   139.6582  35.42156   
18      カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ  045-123-4049   139.5916  35.56189   
19               カナガワケンズシシズシイッチョウメ  046-123-4036   139.5789  35.29642   
20  カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ  045-123-4033   139.6316  35.54655   
21          トウキョウトコガネイシホンチョウイッチョウメ  042-123-4015   139.5094  35.70018   
22             カナガワケンカワサキシミヤマエクノガワ  044-123-4044   139.5998  35.57693   
26   カナガワケンヨコハマシツヅキクナカガワチュウオウニチョウメ  045-123-4051   139.5758  35.54912   
27                    チバケンサクラシカミシヅ  043-123-4001   140.1452  35.71872   
28      カナガワケンフジサワシツジドウニシカイガンニチョウメ  046-123-4040   139.4466  35.32464   
29                トウキョウトハチオウジシオオツカ  042-123-4026   139.4235  35.63787   
31       カナガワケンカワサキシカワ

---
> P-017: 顧客データ（df_customer）を生年月日（birth_day）で高齢順にソートし、先頭から全項目を10件表示せよ。

In [3]:
df_customer.sort_values(by='birth_day', ascending=True).head(10)

customer_id customer_name gender_cd gender   birth_day  age  \
18817  CS003813000014        村山 菜々美         1     女性  1928-11-26   90   
12328  CS026813000004         吉村 朝陽         1     女性  1928-12-14   90   
15682  CS018811000003         熊沢 美里         1     女性  1929-01-07   90   
15302  CS027803000004         内村 拓郎         0     男性  1929-01-12   90   
1681   CS013801000003         天野 拓郎         0     男性  1929-01-15   90   
7511   CS001814000022         鶴田 里穂         1     女性  1929-01-28   90   
2378   CS016815000002         山元 美紀         1     女性  1929-02-22   90   
4680   CS009815000003         中田 里穂         1     女性  1929-04-08   89   
16070  CS005813000015        金谷 恵梨香         1     女性  1929-04-09   89   
6305   CS012813000013         宇野 南朋         1     女性  1929-04-09   89   

      postal_cd                 address application_store_cd application_date  \
18817  182-0007     東京都調布市菊野台**********               S13003         20160214   
12328  251-0043   神奈川県藤沢市辻堂元町**********               S14026         20150723   
15682  204-0004      東京都清瀬市野塩**********               S13018         20150403   
15302  251-0031  神奈川県藤沢市鵠沼藤が谷**********               S14027         20151227   
1681   274-0824     千葉県船橋市前原東**********               S12013         20160120   
7511   144-0045     東京都大田区南六郷**********               S13001         20161012   
2378   184-0005     東京都小金井市桜町**********               S13016         20150629   
4680   154-0014     東京都世田谷区新町**********               S13009         20150421   
16070  165-0032      東京都中野区鷺宮**********               S13005         20150506   
6305   231-0806  神奈川県横浜市中区本牧町**********               S14012         20150712   

          status_cd  
18817  0-00000000-0  
12328  0-00000000-0  
15682  0-00000000-0  
15302  0-00000000-0  
1681   0-00000000-0  
7511   A-20090415-7  
2378   C-20090923-C  
4680   D-20091021-E  
16070  0-00000000-0  
6305   0-00000000-0

---
> P-018: 顧客データ（df_customer）を生年月日（birth_day）で若い順にソートし、先頭から全項目を10件表示せよ。

In [4]:
df_customer.sort_values(by='birth_day', ascending=False).head(10)

customer_id customer_name gender_cd gender   birth_day  age  \
15639  CS035114000004         大村 美里         1     女性  2007-11-25   11   
7468   CS022103000002        福山 はじめ         9     不明  2007-10-02   11   
10745  CS002113000009        柴田 真悠子         1     女性  2007-09-17   11   
19811  CS004115000014         松井 京子         1     女性  2007-08-09   11   
7039   CS002114000010          山内 遥         1     女性  2007-06-03   11   
3670   CS025115000002         小柳 夏希         1     女性  2007-04-18   11   
12493  CS002113000025        広末 まなみ         1     女性  2007-03-30   12   
15977  CS033112000003         長野 美紀         1     女性  2007-03-22   12   
5716   CS007115000006          福岡 瞬         1     女性  2007-03-10   12   
15097  CS014113000008         矢口 莉緒         1     女性  2007-03-05   12   

      postal_cd                  address application_store_cd  \
15639  156-0053       東京都世田谷区桜**********               S13035   
7468   249-0006      神奈川県逗子市逗子**********               S14022   
10745  184-0014    東京都小金井市貫井南町**********               S13002   
19811  165-0031      東京都中野区上鷺宮**********               S13004   
7039   184-0015    東京都小金井市貫井北町**********               S13002   
3670   245-0018  神奈川県横浜市泉区上飯田町**********               S14025   
12493  184-0015    東京都小金井市貫井北町**********               S13002   
15977  245-0051  神奈川県横浜市戸塚区名瀬町**********               S14033   
5716   285-0845      千葉県佐倉市西志津**********               S12007   
15097  260-0041   千葉県千葉市中央区東千葉**********               S12014   

      application_date     status_cd  
15639         20150619  6-20091205-6  
7468          20160909  0-00000000-0  
10745         20160304  0-00000000-0  
19811         20161120  1-20081231-1  
7039          20160920  6-20100510-1  
3670          20160116  D-20100913-D  
12493         20171030  0-00000000-0  
15977         20150606  0-00000000-0  
5716          20151118  F-20101016-F  
15097         20150622  3-20091108-6

---
> P-019: レシート明細データ（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭から10件表示せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合は同一順位を付与するものとする。

In [5]:
df_receipt.head(3)

sales_ymd  sales_epoch store_cd  receipt_no  receipt_sub_no  \
0   20181103   1541203200   S14006         112               1   
1   20181118   1542499200   S13008        1132               2   
2   20170712   1499817600   S14028        1102               1   

      customer_id  product_cd  quantity  amount  
0  CS006214000001  P070305012         1     158  
1  CS008415000097  P070701017         1      81  
2  CS028414000014  P060101005         1     170

In [13]:
df_receipt['amount_rank'] = df_receipt['amount'].rank(ascending=False, method='min')
df_receipt[['customer_id', 'amount', 'amount_rank']].sort_values(by='amount_rank', ascending=True).head(10)

customer_id  amount  amount_rank
1202   CS011415000006   10925          1.0
62317  ZZ000000000000    6800          2.0
54095  CS028605000002    5780          3.0
4632   CS015515000034    5480          4.0
72747  ZZ000000000000    5480          4.0
10320  ZZ000000000000    5480          4.0
97294  CS021515000089    5440          7.0
28304  ZZ000000000000    5440          7.0
92246  CS009415000038    5280          9.0
68553  CS040415000200    5280          9.0

---
> P-020: レシート明細データ（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭から10件表示せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合でも別順位を付与すること。

In [3]:
df_receipt['amount_rank'] = df_receipt.amount.rank(ascending=False, method='first')
df_receipt[['customer_id', 'amount', 'amount_rank']].sort_values(by='amount_rank', ascending=True).head(10)

customer_id  amount  amount_rank
1202   CS011415000006   10925          1.0
62317  ZZ000000000000    6800          2.0
54095  CS028605000002    5780          3.0
4632   CS015515000034    5480          4.0
10320  ZZ000000000000    5480          5.0
72747  ZZ000000000000    5480          6.0
28304  ZZ000000000000    5440          7.0
97294  CS021515000089    5440          8.0
596    CS015515000083    5280          9.0
11275  CS017414000114    5280         10.0

---
> P-021: レシート明細データ（df_receipt）に対し、件数をカウントせよ。

In [4]:
len(df_receipt)

104681

---
> P-022: レシート明細データ（df_receipt）の顧客ID（customer_id）に対し、ユニーク件数をカウントせよ。

In [5]:
df_receipt.customer_id.nunique()

8307

---
> P-023: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）と売上数量（quantity）を合計せよ。

In [7]:
df_receipt.groupby('store_cd')[['amount', 'quantity']].sum().head()

amount  quantity
store_cd                  
S12007    638761      2099
S12013    787513      2425
S12014    725167      2358
S12029    794741      2555
S12030    684402      2403

In [10]:
df_receipt.groupby('store_cd').agg({'amount': 'sum', 'quantity': 'sum'}).reset_index().head()

store_cd  amount  quantity
0   S12007  638761      2099
1   S12013  787513      2425
2   S12014  725167      2358
3   S12029  794741      2555
4   S12030  684402      2403

In [12]:
df_receipt.groupby('store_cd').agg(amount_sum=('amount', 'sum'), quantity_sum=('quantity', 'sum')).reset_index().head()

store_cd  amount_sum  quantity_sum
0   S12007      638761          2099
1   S12013      787513          2425
2   S12014      725167          2358
3   S12029      794741          2555
4   S12030      684402          2403

---
> P-024: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上年月日（sales_ymd）を求め、10件表示せよ。

In [14]:
df_receipt.groupby('customer_id').agg(sales_ymd_latest=('sales_ymd', 'last')).head() # NG(lastがlatestとは限らない)

sales_ymd_latest
customer_id                     
CS001113000004          20190308
CS001114000005          20190731
CS001115000010          20171228
CS001205000004          20190625
CS001205000006          20180207

In [15]:
df_receipt.groupby('customer_id').agg(sales_ymd_latest=('sales_ymd', 'max')).head()

sales_ymd_latest
customer_id                     
CS001113000004          20190308
CS001114000005          20190731
CS001115000010          20190405
CS001205000004          20190625
CS001205000006          20190224

---
> P-025: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに最も古い売上年月日（sales_ymd）を求め、10件表示せよ。

In [2]:
df_receipt.groupby('customer_id').agg(sales_ymd_oldest=('sales_ymd', 'min')).reset_index().head()

customer_id  sales_ymd_oldest
0  CS001113000004          20190308
1  CS001114000005          20180503
2  CS001115000010          20171228
3  CS001205000004          20170914
4  CS001205000006          20180207

---
> P-026: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上年月日（sales_ymd）と古い売上年月日を求め、両者が異なるデータを10件表示せよ。

In [9]:
dfg = df_receipt.groupby('customer_id').agg(sales_ymd_latest=('sales_ymd', 'max'), sales_ymd_oldest=('sales_ymd', 'min')).reset_index()
dfg['multiple_sales'] = dfg['sales_ymd_latest'] != dfg['sales_ymd_oldest']
# dfg[dfg['multiple_sales']].head()
dfg.query('multiple_sales == True').head()

customer_id  sales_ymd_latest  sales_ymd_oldest  multiple_sales
1   CS001114000005          20190731          20180503            True
2   CS001115000010          20190405          20171228            True
3   CS001205000004          20190625          20170914            True
4   CS001205000006          20190224          20180207            True
13  CS001214000009          20190902          20170306            True

---
> P-027: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、降順でTOP5を表示せよ。

In [12]:
df_receipt.groupby('store_cd').agg(amount_mean=('amount', 'mean')).sort_values(by='amount_mean', ascending=False).head()

amount_mean
store_cd             
S13052     402.867470
S13015     351.111960
S13003     350.915519
S14010     348.791262
S13001     348.470386

---
> P-028: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の中央値を計算し、降順でTOP5を表示せよ。

In [14]:
df_receipt.groupby(by='store_cd').agg(amount_median=('amount', 'median')).sort_values(by='amount_median',ascending=False).head()

amount_median
store_cd               
S13052            190.0
S14010            188.0
S14050            185.0
S14040            180.0
S13003            180.0

---
> P-029: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに商品コード（product_cd）の最頻値を求め、10件表示させよ。

In [15]:
# aggでmodeはできないのでコケる
df_receipt.groupby(by='store_cd').agg(product_cd_mode=('product_cd', 'mode')).head()

AttributeError: 'SeriesGroupBy' object has no attribute 'mode'

In [18]:
df_receipt.groupby(by='store_cd')['product_cd']

In [21]:
# groupbyからまず1列分を選べる、それにapplyで関数適用して1つの値にする。そしたらそれらをgroupの値としてまとめてくれる
df_receipt.groupby(by='store_cd')['product_cd'].apply(lambda x: x.mode()).head(10)

store_cd   
S12007    0    P060303001
S12013    0    P060303001
S12014    0    P060303001
S12029    0    P060303001
S12030    0    P060303001
S13001    0    P060303001
S13002    0    P060303001
S13003    0    P071401001
S13004    0    P060303001
S13005    0    P040503001
Name: product_cd, dtype: object

In [22]:
# SeriesGroupbyはmode直計算できない(でも分散と標準偏差はできる。↓)
df_receipt.groupby(by='store_cd')['product_cd'].mode().head(10)

AttributeError: 'SeriesGroupBy' object has no attribute 'mode'

---
> P-030: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の分散を計算し、降順で5件表示せよ。

In [28]:
df_receipt.groupby(by='store_cd')['amount'].var(ddof=0).reset_index().head(2)#.sort_values('amount', ascending=False).head()

store_cd         amount
0   S12007  199878.572908
1   S12013  221059.615563

In [29]:
df_receipt.groupby(by='store_cd')['amount'].var(ddof=0).reset_index().sort_values('amount', ascending=False).head()

store_cd         amount
28   S13052  440088.701311
31   S14011  306314.558164
42   S14034  296920.081011
5    S13001  295431.993329
12   S13015  295294.361116

In [31]:
# これじゃ値が一致しない
df_receipt.groupby(by='store_cd').agg(amount_var=('amount', np.var)).reset_index().sort_values('amount_var', ascending=False).head()

store_cd     amount_var
28   S13052  441863.252526
31   S14011  306442.242432
42   S14034  297068.392740
5    S13001  295558.842618
12   S13015  295427.197086

---
> P-031: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の標準偏差を計算し、降順で5件表示せよ。

TIPS:

PandasとNumpyでddofのデフォルト値が異なることに注意しましょう
```
Pandas：
DataFrame.std(self, axis=None, skipna=None, level=None, ddof=1, numeric_only=None, **kwargs)
Numpy:
numpy.std(a, axis=None, dtype=None, out=None, ddof=0, keepdims=)
```

In [32]:
df_receipt.groupby(by='store_cd')['amount'].std(ddof=0).reset_index().sort_values('amount', ascending=False).head()

store_cd      amount
28   S13052  663.391816
31   S14011  553.456916
42   S14034  544.903736
5    S13001  543.536561
12   S13015  543.409938

---
> P-032: レシート明細データ（df_receipt）の売上金額（amount）について、25％刻みでパーセンタイル値を求めよ。

In [4]:
df_receipt['amount'].quantile([.25, .5, .75])

0.25    102.0
0.50    170.0
0.75    288.0
Name: amount, dtype: float64

---
> P-033: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、330以上のものを抽出せよ。

In [6]:
df_receipt.groupby('store_cd').agg(amount_mean=('amount', 'mean')).query("amount_mean >= 330")

amount_mean
store_cd             
S12013     330.194130
S13001     348.470386
S13003     350.915519
S13004     330.943949
S13015     351.111960
S13019     330.208616
S13020     337.879932
S13052     402.867470
S14010     348.791262
S14011     335.718333
S14026     332.340588
S14045     330.082073
S14047     330.077073

---
> P-034: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求めよ。ただし、顧客IDが"Z"から始まるものは非会員を表すため、除外して計算すること。

In [14]:
df_receipt.query("customer_id.str.startswith('Z') == False").groupby('customer_id').agg(amount_sum=('amount', 'sum')).mean()

amount_sum    2547.742235
dtype: float64

---
> P-035: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求め、平均以上に買い物をしている顧客を抽出し、10件表示せよ。ただし、顧客IDが"Z"から始まるものは非会員を表すため、除外して計算すること。

In [13]:
dfg = df_receipt.query("customer_id.str.startswith('Z') == False").groupby('customer_id').agg(amount_sum=('amount', 'sum'))
mean = dfg.mean().iloc[0] # Seriesをqueryに突っ込んだらmultiple lineクエリはダメみたいなこと言っておこられる
dfg.query(f"amount_sum >= {mean}").head()


amount_sum
customer_id               
CS001115000010        3044
CS001205000006        3337
CS001214000009        4685
CS001214000017        4132
CS001214000052        5639

---
> P-036: レシート明細データ（df_receipt）と店舗データ（df_store）を内部結合し、レシート明細データの全項目と店舗データの店舗名（store_name）を10件表示せよ。

In [15]:
df_receipt.merge(df_store[['store_cd', 'store_name']], on='store_cd', how='inner').head()

sales_ymd  sales_epoch store_cd  receipt_no  receipt_sub_no  \
0   20181103   1541203200   S14006         112               1   
1   20181116   1542326400   S14006         112               2   
2   20170118   1484697600   S14006        1162               1   
3   20190524   1558656000   S14006        1192               1   
4   20190419   1555632000   S14006         112               2   

      customer_id  product_cd  quantity  amount store_name  
0  CS006214000001  P070305012         1     158       葛が谷店  
1  ZZ000000000000  P080401001         1      48       葛が谷店  
2  CS006815000006  P050406035         1     220       葛が谷店  
3  CS006514000034  P060104003         1      80       葛が谷店  
4  ZZ000000000000  P060501002         1     148       葛が谷店

---
> P-037: 商品データ（df_product）とカテゴリデータ（df_category）を内部結合し、商品データの全項目とカテゴリデータのカテゴリ小区分名（category_small_name）を10件表示せよ。

In [19]:
df_category.head(2)

category_major_cd category_major_name category_medium_cd  \
0                04                  惣菜               0401   
1                04                  惣菜               0401   

  category_medium_name category_small_cd category_small_name  
0                  御飯類            040101                 弁当類  
1                  御飯類            040102                 寿司類

In [18]:
df_product.head(2)

product_cd category_major_cd category_medium_cd category_small_cd  \
0  P040101001                04               0401            040101   
1  P040101002                04               0401            040101   

   unit_price  unit_cost  
0       198.0      149.0  
1       218.0      164.0

In [21]:
df_product.merge(df_category[['category_major_cd', 'category_medium_cd', 'category_small_cd', 'category_small_name']], on=['category_major_cd', 'category_medium_cd', 'category_small_cd'], how='inner').head()

product_cd category_major_cd category_medium_cd category_small_cd  \
0  P040101001                04               0401            040101   
1  P040101002                04               0401            040101   
2  P040101003                04               0401            040101   
3  P040101004                04               0401            040101   
4  P040101005                04               0401            040101   

   unit_price  unit_cost category_small_name  
0       198.0      149.0                 弁当類  
1       218.0      164.0                 弁当類  
2       230.0      173.0                 弁当類  
3       248.0      186.0                 弁当類  
4       268.0      201.0                 弁当類

---
> P-038: 顧客データ（df_customer）とレシート明細データ（df_receipt）から、顧客ごとの売上金額合計を求め、10件表示せよ。ただし、売上実績がない顧客については売上金額を0として表示させること。また、顧客は性別コード（gender_cd）が女性（1）であるものを対象とし、非会員（顧客IDが"Z"から始まるもの）は除外すること。

In [2]:
df_customer.head(2)

customer_id customer_name gender_cd gender   birth_day  age postal_cd  \
0  CS021313000114        大野 あや子         1     女性  1981-04-29   37  259-1113   
1  CS037613000071         六角 雅彦         9     不明  1952-04-01   66  136-0076   

                address application_store_cd application_date     status_cd  
0  神奈川県伊勢原市粟窪**********               S14021         20150905  0-00000000-0  
1    東京都江東区南砂**********               S13037         20150414  0-00000000-0

In [3]:
df_receipt.head(2)

sales_ymd  sales_epoch store_cd  receipt_no  receipt_sub_no  \
0   20181103   1541203200   S14006         112               1   
1   20181118   1542499200   S13008        1132               2   

      customer_id  product_cd  quantity  amount  
0  CS006214000001  P070305012         1     158  
1  CS008415000097  P070701017         1      81

In [13]:
df_receipt_agg = df_receipt.groupby('customer_id').agg(amount_sum=('amount', 'sum')).reset_index()
df_customer.query("gender_cd == '1' and (customer_id.str.startswith('Z') == False)").merge(df_receipt_agg, on='customer_id', how='left').fillna({'amount_sum': 0}).head()

customer_id customer_name gender_cd gender   birth_day  age postal_cd  \
0  CS021313000114        大野 あや子         1     女性  1981-04-29   37  259-1113   
1  CS031415000172       宇多田 貴美子         1     女性  1976-10-04   42  151-0053   
2  CS028811000001        堀井 かおり         1     女性  1933-03-27   86  245-0016   
3  CS001215000145         田崎 美紀         1     女性  1995-03-29   24  144-0055   
4  CS015414000103         奥野 陽子         1     女性  1977-08-09   41  136-0073   

                  address application_store_cd application_date     status_cd  \
0    神奈川県伊勢原市粟窪**********               S14021         20150905  0-00000000-0   
1     東京都渋谷区代々木**********               S13031         20150529  D-20100325-C   
2  神奈川県横浜市泉区和泉町**********               S14028         20160115  0-00000000-0   
3     東京都大田区仲六郷**********               S13001         20170605  6-20090929-2   
4      東京都江東区北砂**********               S13015         20150722  B-20100609-B   

   amount_sum  
0         0.0  
1      5088.0  
2         0.0  
3       875.0  
4      3122.0

---
> P-039: レシート明細データ（df_receipt）から、売上日数の多い顧客の上位20件を抽出したデータと、売上金額合計の多い顧客の上位20件を抽出したデータをそれぞれ作成し、さらにその2つを完全外部結合せよ。ただし、非会員（顧客IDが"Z"から始まるもの）は除外すること。

In [16]:
df = df_receipt.merge(df_customer.query("customer_id.str.startswith('Z') == False"), on='customer_id', how='inner')
dfa1 = df.groupby('customer_id').agg(sales_days=('sales_ymd', 'nunique')).reset_index().sort_values('sales_days', ascending=False).head(20)
dfa2 = df.groupby('customer_id').agg(amount_sum=('amount', 'sum')).reset_index().sort_values('amount_sum', ascending=False).head(20)
dfa = dfa1.merge(dfa2, on='customer_id', how='outer')
dfa

customer_id  sales_days  amount_sum
0   CS040214000008        23.0         NaN
1   CS015415000185        22.0     20153.0
2   CS010214000010        22.0     18585.0
3   CS010214000002        21.0         NaN
4   CS028415000007        21.0     19127.0
5   CS017415000097        20.0     23086.0
6   CS016415000141        20.0     18372.0
7   CS031414000051        19.0     19202.0
8   CS014214000023        19.0         NaN
9   CS022515000226        19.0         NaN
10  CS021515000172        19.0         NaN
11  CS039414000052        19.0         NaN
12  CS021514000045        19.0         NaN
13  CS022515000028        18.0         NaN
14  CS030214000008        18.0         NaN
15  CS021515000056        18.0         NaN
16  CS014415000077        18.0         NaN
17  CS021515000211        18.0         NaN
18  CS032415000209        18.0         NaN
19  CS031414000073        18.0         NaN
20  CS001605000009         NaN     18925.0
21  CS006515000023         NaN     18372.0
22  CS011414000106         NaN     18338.0
23  CS038415000104         NaN     17847.0
24  CS035414000024         NaN     17615.0
25  CS021515000089         NaN     17580.0
26  CS032414000072         NaN     16563.0
27  CS016415000101         NaN     16348.0
28  CS011415000006         NaN     16094.0
29  CS034415000047         NaN     16083.0
30  CS007514000094         NaN     15735.0
31  CS009414000059         NaN     15492.0
32  CS030415000034         NaN     15468.0
33  CS015515000034         NaN     15300.0

---
> P-040: 全ての店舗と全ての商品を組み合わせたデータを作成したい。店舗データ（df_store）と商品データ（df_product）を直積し、件数を計算せよ。

In [2]:
df_store.head(2)

store_cd store_name prefecture_cd prefecture          address  \
0   S12014       千草台店            12        千葉県  千葉県千葉市稲毛区千草台一丁目   
1   S13002       国分寺店            13        東京都     東京都国分寺市本多二丁目   

             address_kana        tel_no  longitude  latitude  floor_area  
0  チバケンチバシイナゲクチグサダイイッチョウメ  043-123-4003   140.1180  35.63559      1698.0  
1    トウキョウトコクブンジシホンダニチョウメ  042-123-4008   139.4802  35.70566      1735.0

In [3]:
df_product.head(2)

product_cd category_major_cd category_medium_cd category_small_cd  \
0  P040101001                04               0401            040101   
1  P040101002                04               0401            040101   

   unit_price  unit_cost  
0       198.0      149.0  
1       218.0      164.0

In [4]:
df_store_temp = df_store.copy()
df_product_temp = df_product.copy()
df_store_temp['key'] = None
df_product_temp['key'] = None
len(df_store_temp.merge(df_product_temp, on='key', how='inner'))

531590

---
> P-041: レシート明細データ（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、前回売上があった日からの売上金額増減を計算せよ。そして結果を10件表示せよ。

In [9]:
dfa = df_receipt.groupby('sales_ymd').agg(amount_sum=('amount', 'sum'))
dfa['diff'] = dfa.diff()
dfa.head()


amount_sum    diff
sales_ymd                    
20170101        33723     NaN
20170102        24165 -9558.0
20170103        27503  3338.0
20170104        36165  8662.0
20170105        37830  1665.0

---
> P-042: レシート明細データ（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、各日付のデータに対し、前回、前々回、3回前に売上があった日のデータを結合せよ。そして結果を10件表示せよ。

In [5]:
dfa = df_receipt.groupby('sales_ymd').agg(amount_sum=('amount', 'sum'))
dfa['amount_sum_before_1'] = dfa.amount_sum.shift(1)
dfa['amount_sum_before_2'] = dfa.amount_sum.shift(2)
dfa['amount_sum_before_3'] = dfa.amount_sum.shift(3)
dfa.head(10)

amount_sum  amount_sum_before_1  amount_sum_before_2  \
sales_ymd                                                         
20170101        33723                  NaN                  NaN   
20170102        24165              33723.0                  NaN   
20170103        27503              24165.0              33723.0   
20170104        36165              27503.0              24165.0   
20170105        37830              36165.0              27503.0   
20170106        32387              37830.0              36165.0   
20170107        23415              32387.0              37830.0   
20170108        24737              23415.0              32387.0   
20170109        26718              24737.0              23415.0   
20170110        20143              26718.0              24737.0   

           amount_sum_before_3  
sales_ymd                       
20170101                   NaN  
20170102                   NaN  
20170103                   NaN  
20170104               33723.0  
20170105               24165.0  
20170106               27503.0  
20170107               36165.0  
20170108               37830.0  
20170109               32387.0  
20170110               23415.0

In [10]:
dfa = df_receipt.groupby('sales_ymd').agg(amount_sum=('amount', 'sum')).reset_index()
dfa = pd.concat([dfa, dfa.shift(1).add_suffix('_lag1'), dfa.shift(2).add_suffix('_lag2'), dfa.shift(3).add_suffix('_lag3')], axis=1)
dfa.head(10)

sales_ymd  amount_sum  sales_ymd_lag1  amount_sum_lag1  sales_ymd_lag2  \
0   20170101       33723             NaN              NaN             NaN   
1   20170102       24165      20170101.0          33723.0             NaN   
2   20170103       27503      20170102.0          24165.0      20170101.0   
3   20170104       36165      20170103.0          27503.0      20170102.0   
4   20170105       37830      20170104.0          36165.0      20170103.0   
5   20170106       32387      20170105.0          37830.0      20170104.0   
6   20170107       23415      20170106.0          32387.0      20170105.0   
7   20170108       24737      20170107.0          23415.0      20170106.0   
8   20170109       26718      20170108.0          24737.0      20170107.0   
9   20170110       20143      20170109.0          26718.0      20170108.0   

   amount_sum_lag2  sales_ymd_lag3  amount_sum_lag3  
0              NaN             NaN              NaN  
1              NaN             NaN              NaN  
2          33723.0             NaN              NaN  
3          24165.0      20170101.0          33723.0  
4          27503.0      20170102.0          24165.0  
5          36165.0      20170103.0          27503.0  
6          37830.0      20170104.0          36165.0  
7          32387.0      20170105.0          37830.0  
8          23415.0      20170106.0          32387.0  
9          24737.0      20170107.0          23415.0

---
> P-043： レシート明細データ（df_receipt）と顧客データ（df_customer）を結合し、性別コード（gender_cd）と年代（ageから計算）ごとに売上金額（amount）を合計した売上サマリデータを作成せよ。性別コードは0が男性、1が女性、9が不明を表すものとする。
>
> ただし、項目構成は年代、女性の売上金額、男性の売上金額、性別不明の売上金額の4項目とすること（縦に年代、横に性別のクロス集計）。また、年代は10歳ごとの階級とすること。

In [21]:
df = df_receipt.merge(df_customer, on='customer_id', how='inner')
df['era'] = (df.age // 10) * 10
dfa = df.pivot_table(index='era', columns='gender_cd', values='amount', aggfunc='sum')
dfa.rename(columns={'0': 'male', '1': 'female', '9': 'unknown'}).rename_axis(columns=None).reset_index()

era      male     female   unknown
0   10    1591.0   149836.0    4317.0
1   20   72940.0  1363724.0   44328.0
2   30  177322.0   693047.0   50441.0
3   40   19355.0  9320791.0  483512.0
4   50   54320.0  6685192.0  342923.0
5   60  272469.0   987741.0   71418.0
6   70   13435.0    29764.0    2427.0
7   80   46360.0   262923.0    5111.0
8   90       NaN     6260.0       NaN

---
> P-044： 043で作成した売上サマリデータ（df_sales_summary）は性別の売上を横持ちさせたものであった。このデータから性別を縦持ちさせ、年代、性別コード、売上金額の3項目に変換せよ。ただし、性別コードは男性を"00"、女性を"01"、不明を"99"とする。

In [33]:
dfa.rename(columns={col: col.zfill(2) for col in dfa.columns}).rename_axis(columns=None).reset_index().melt(id_vars=['era'], value_vars=['00', '01', '09'], var_name='gender_cd', value_name='amount_sum').sort_values(['era', 'gender_cd'])

era gender_cd  amount_sum
0    10        00      1591.0
9    10        01    149836.0
18   10        09      4317.0
1    20        00     72940.0
10   20        01   1363724.0
19   20        09     44328.0
2    30        00    177322.0
11   30        01    693047.0
20   30        09     50441.0
3    40        00     19355.0
12   40        01   9320791.0
21   40        09    483512.0
4    50        00     54320.0
13   50        01   6685192.0
22   50        09    342923.0
5    60        00    272469.0
14   60        01    987741.0
23   60        09     71418.0
6    70        00     13435.0
15   70        01     29764.0
24   70        09      2427.0
7    80        00     46360.0
16   80        01    262923.0
25   80        09      5111.0
8    90        00         NaN
17   90        01      6260.0
26   90        09         NaN

---
> P-045: 顧客データ（df_customer）の生年月日（birth_day）は日付型でデータを保有している。これをYYYYMMDD形式の文字列に変換し、顧客ID（customer_id）とともに10件表示せよ。

In [40]:
df = df_customer.copy()
df['birth_day'] = pd.to_datetime(df.birth_day)
df['birth_day_formated'] = df.birth_day.dt.strftime('%Y%m%d')
df.head()

customer_id customer_name gender_cd gender  birth_day  age postal_cd  \
0  CS021313000114        大野 あや子         1     女性 1981-04-29   37  259-1113   
1  CS037613000071         六角 雅彦         9     不明 1952-04-01   66  136-0076   
2  CS031415000172       宇多田 貴美子         1     女性 1976-10-04   42  151-0053   
3  CS028811000001        堀井 かおり         1     女性 1933-03-27   86  245-0016   
4  CS001215000145         田崎 美紀         1     女性 1995-03-29   24  144-0055   

                  address application_store_cd application_date     status_cd  \
0    神奈川県伊勢原市粟窪**********               S14021         20150905  0-00000000-0   
1      東京都江東区南砂**********               S13037         20150414  0-00000000-0   
2     東京都渋谷区代々木**********               S13031         20150529  D-20100325-C   
3  神奈川県横浜市泉区和泉町**********               S14028         20160115  0-00000000-0   
4     東京都大田区仲六郷**********               S13001         20170605  6-20090929-2   

  birth_day_formated  
0           19810429  
1           19520401  
2           19761004  
3           19330327  
4           19950329

---
> P-046: 顧客データ（df_customer）の申し込み日（application_date）はYYYYMMDD形式の文字列型でデータを保有している。これを日付型に変換し、顧客ID（customer_id）とともに10件表示せよ。

In [41]:
df_customer['application_date'] = pd.to_datetime(df_customer.application_date)
df_customer[['customer_id', 'application_date']].head()

customer_id application_date
0  CS021313000114       2015-09-05
1  CS037613000071       2015-04-14
2  CS031415000172       2015-05-29
3  CS028811000001       2016-01-15
4  CS001215000145       2017-06-05

In [42]:
pd.concat([df_customer.customer_id, pd.to_datetime(df_customer.application_date)], axis=1).head()

customer_id application_date
0  CS021313000114       2015-09-05
1  CS037613000071       2015-04-14
2  CS031415000172       2015-05-29
3  CS028811000001       2016-01-15
4  CS001215000145       2017-06-05

---
> P-047: レシート明細データ（df_receipt）の売上日（sales_ymd）はYYYYMMDD形式の数値型でデータを保有している。これを日付型に変換し、レシート番号（receipt_no）、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。

In [44]:
pd.concat([df_receipt.receipt_no, df_receipt.receipt_sub_no, pd.to_datetime(df_receipt.sales_ymd)], axis=1).head()

receipt_no  receipt_sub_no                     sales_ymd
0         112               1 1970-01-01 00:00:00.020181103
1        1132               2 1970-01-01 00:00:00.020181118
2        1102               1 1970-01-01 00:00:00.020170712
3        1132               1 1970-01-01 00:00:00.020190205
4        1102               2 1970-01-01 00:00:00.020180821

---
> P-048: レシート明細データ（df_receipt）の売上エポック秒（sales_epoch）は数値型のUNIX秒でデータを保有している。これを日付型に変換し、レシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。

In [50]:
pd.concat([df_receipt.receipt_no, df_receipt.receipt_sub_no, pd.to_datetime(df_receipt.sales_epoch, unit='s')], axis=1).rename(columns={'sales_epoch': 'sales_ymd'}).head()

receipt_no  receipt_sub_no  sales_ymd
0         112               1 2018-11-03
1        1132               2 2018-11-18
2        1102               1 2017-07-12
3        1132               1 2019-02-05
4        1102               2 2018-08-21

---
> P-049: レシート明細データ（df_receipt）の売上エポック秒（sales_epoch）を日付型に変換し、「年」だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。

In [53]:
pd.concat([df_receipt.receipt_no, df_receipt.receipt_sub_no, pd.to_datetime(df_receipt.sales_epoch, unit='s').dt.year], axis=1).rename(columns={'sales_epoch': 'sales_year'}).head()

receipt_no  receipt_sub_no  sales_year
0         112               1        2018
1        1132               2        2018
2        1102               1        2017
3        1132               1        2019
4        1102               2        2018

In [54]:
pd.concat([df_receipt.receipt_no, df_receipt.receipt_sub_no, pd.to_datetime(df_receipt.sales_epoch, unit='s').dt.year.rename('sales_year')], axis=1).head()

receipt_no  receipt_sub_no  sales_year
0         112               1        2018
1        1132               2        2018
2        1102               1        2017
3        1132               1        2019
4        1102               2        2018

---
> P-050: レシート明細データ（df_receipt）の売上エポック秒（sales_epoch）を日付型に変換し、「月」だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。なお、「月」は0埋め2桁で取り出すこと。

In [5]:
pd.concat([df_receipt.receipt_no, df_receipt.receipt_sub_no, pd.to_datetime(df_receipt.sales_epoch, unit='s').dt.month.rename('sales_month').astype(str).str.zfill(2)], axis=1)

receipt_no  receipt_sub_no sales_month
0              112               1          11
1             1132               2          11
2             1102               1          07
3             1132               1          02
4             1102               2          08
...            ...             ...         ...
104676        1132               2          02
104677        1132               2          09
104678        1122               1          03
104679        1142               1          03
104680        1102               2          04

[104681 rows x 3 columns]

In [6]:
pd.concat([df_receipt.receipt_no, df_receipt.receipt_sub_no, pd.to_datetime(df_receipt.sales_epoch, unit='s').dt.strftime('%m').rename('sales_month')], axis=1)

receipt_no  receipt_sub_no sales_month
0              112               1          11
1             1132               2          11
2             1102               1          07
3             1132               1          02
4             1102               2          08
...            ...             ...         ...
104676        1132               2          02
104677        1132               2          09
104678        1122               1          03
104679        1142               1          03
104680        1102               2          04

[104681 rows x 3 columns]

---
> P-051: レシート明細データ（df_receipt）の売上エポック秒を日付型に変換し、「日」だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。なお、「日」は0埋め2桁で取り出すこと。

In [4]:
pd.concat([df_receipt.receipt_no, df_receipt.receipt_sub_no, pd.to_datetime(df_receipt.sales_epoch, unit='s').dt.strftime('%d').rename('sales_day')], axis=1).head()

receipt_no  receipt_sub_no sales_day
0         112               1        03
1        1132               2        18
2        1102               1        12
3        1132               1        05
4        1102               2        21

---
> P-052: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計の上、売上金額合計に対して2,000円以下を0、2,000円より大きい金額を1に二値化し、顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [5]:
df = df_receipt.query("customer_id.str.startswith('Z') == False").groupby('customer_id').agg(amount_sum=('amount', 'sum')).reset_index()
df['amount_2000over'] = (df.amount_sum > 2000).astype(int)
df.head()

customer_id  amount_sum  amount_2000over
0  CS001113000004        1298                0
1  CS001114000005         626                0
2  CS001115000010        3044                1
3  CS001205000004        1988                0
4  CS001205000006        3337                1

In [6]:
df = df_receipt.query("customer_id.str.startswith('Z') == False").groupby('customer_id').agg(amount_sum=('amount', 'sum')).reset_index()
df['amount_2000over'] = df.amount_sum.apply(lambda x: 1 if x > 2000 else 0)
df.head()

customer_id  amount_sum  amount_2000over
0  CS001113000004        1298                0
1  CS001114000005         626                0
2  CS001115000010        3044                1
3  CS001205000004        1988                0
4  CS001205000006        3337                1

---
> P-053: 顧客データ（df_customer）の郵便番号（postal_cd）に対し、東京（先頭3桁が100〜209のもの）を1、それ以外のものを0に二値化せよ。さらにレシート明細データ（df_receipt）と結合し、全期間において売上実績のある顧客数を、作成した二値ごとにカウントせよ。

In [7]:
df_customer['is_tokyo'] = df_customer.postal_cd.str[:3].astype(int).apply(lambda x: x >= 100 and x <= 209)
display(df_customer.head())
df_customer.merge(df_receipt, on='customer_id', how='inner').groupby('is_tokyo').agg(customer_id_count=('customer_id', 'nunique'))

customer_id customer_name gender_cd gender   birth_day  age postal_cd  \
0  CS021313000114        大野 あや子         1     女性  1981-04-29   37  259-1113   
1  CS037613000071         六角 雅彦         9     不明  1952-04-01   66  136-0076   
2  CS031415000172       宇多田 貴美子         1     女性  1976-10-04   42  151-0053   
3  CS028811000001        堀井 かおり         1     女性  1933-03-27   86  245-0016   
4  CS001215000145         田崎 美紀         1     女性  1995-03-29   24  144-0055   

                  address application_store_cd application_date     status_cd  \
0    神奈川県伊勢原市粟窪**********               S14021         20150905  0-00000000-0   
1      東京都江東区南砂**********               S13037         20150414  0-00000000-0   
2     東京都渋谷区代々木**********               S13031         20150529  D-20100325-C   
3  神奈川県横浜市泉区和泉町**********               S14028         20160115  0-00000000-0   
4     東京都大田区仲六郷**********               S13001         20170605  6-20090929-2   

   is_tokyo  
0     False  
1      True  
2      True  
3     False  
4      True

customer_id_count
is_tokyo                   
False                  3906
True                   4400

---
> P-054: 顧客データ（df_customer）の住所（address）は、埼玉県、千葉県、東京都、神奈川県のいずれかとなっている。都道府県毎にコード値を作成し、顧客ID、住所とともに10件表示せよ。値は埼玉県を11、千葉県を12、東京都を13、神奈川県を14とすること。

In [5]:
df_customer['address_code'] = df_customer.address.replace({r'埼玉県': 11, r'千葉県': 12, r'東京都': 13, r'神奈川県': 14}, regex=True)
df_customer[['customer_id', 'address', 'address_code']].head()

customer_id                 address  address_code
0  CS021313000114    神奈川県伊勢原市粟窪**********            14
1  CS037613000071      東京都江東区南砂**********            13
2  CS031415000172     東京都渋谷区代々木**********            13
3  CS028811000001  神奈川県横浜市泉区和泉町**********            14
4  CS001215000145     東京都大田区仲六郷**********            13

In [13]:
df_customer['address_code'] = df_customer.address.str.extract(r'(.+?[都道府県])', expand=False).map({'埼玉県': 11, '千葉県': 12, '東京都': 13, '神奈川県': 14})
df_customer[['customer_id', 'address', 'address_code']].head()

customer_id                 address  address_code
0  CS021313000114    神奈川県伊勢原市粟窪**********            14
1  CS037613000071      東京都江東区南砂**********            13
2  CS031415000172     東京都渋谷区代々木**********            13
3  CS028811000001  神奈川県横浜市泉区和泉町**********            14
4  CS001215000145     東京都大田区仲六郷**********            13

---
> P-055: レシート明細（df_receipt）データの売上金額（amount）を顧客ID（customer_id）ごとに合計し、その合計金額の四分位点を求めよ。その上で、顧客ごとの売上金額合計に対して以下の基準でカテゴリ値を作成し、顧客ID、売上金額合計とともに10件表示せよ。カテゴリ値は順に1〜4とする。
>
> - 最小値以上第1四分位未満 ・・・ 1を付与
> - 第1四分位以上第2四分位未満 ・・・ 2を付与
> - 第2四分位以上第3四分位未満 ・・・ 3を付与
> - 第3四分位以上 ・・・ 4を付与

In [5]:
df_customer2 = df_receipt.groupby('customer_id').agg(amount_sum=('amount', 'sum')).reset_index()
display(df_customer2.head(2))
q25 = df_customer2.amount_sum.quantile(0.25)
q50 = df_customer2.amount_sum.quantile(0.5)
q75 = df_customer2.amount_sum.quantile(0.75)
df_customer2['amount_sum_bin'] = pd.cut(df_customer2.amount_sum, bins=[df_customer2.amount_sum.min() - 1, q25, q50, q75, df_customer2.amount_sum.max() + 1], labels=[1,2,3,4], right=False)
df_customer2.head(10)
#pd.cut(df_receipt.amoun, 

customer_id  amount_sum
0  CS001113000004        1298
1  CS001114000005         626

customer_id  amount_sum amount_sum_bin
0  CS001113000004        1298              2
1  CS001114000005         626              2
2  CS001115000010        3044              3
3  CS001205000004        1988              3
4  CS001205000006        3337              3
5  CS001211000025         456              1
6  CS001212000027         448              1
7  CS001212000031         296              1
8  CS001212000046         228              1
9  CS001212000070         456              1

---
> P-056: 顧客データ（df_customer）の年齢（age）をもとに10歳刻みで年代を算出し、顧客ID（customer_id）、生年月日（birth_day）とともに10件表示せよ。ただし、60歳以上は全て60歳代とすること。年代を表すカテゴリ名は任意とする。

In [12]:
df_customer['era'] = pd.cut(df_customer.age, bins=[0, 10, 20, 30, 40, 50 ,60, df_customer.age.max() + 1], right=False, labels=[0, 10, 20, 30, 40, 50, 60])
df_customer.head()

customer_id customer_name gender_cd gender   birth_day  age postal_cd  \
0  CS021313000114        大野 あや子         1     女性  1981-04-29   37  259-1113   
1  CS037613000071         六角 雅彦         9     不明  1952-04-01   66  136-0076   
2  CS031415000172       宇多田 貴美子         1     女性  1976-10-04   42  151-0053   
3  CS028811000001        堀井 かおり         1     女性  1933-03-27   86  245-0016   
4  CS001215000145         田崎 美紀         1     女性  1995-03-29   24  144-0055   

                  address application_store_cd  application_date  \
0    神奈川県伊勢原市粟窪**********               S14021          20150905   
1      東京都江東区南砂**********               S13037          20150414   
2     東京都渋谷区代々木**********               S13031          20150529   
3  神奈川県横浜市泉区和泉町**********               S14028          20160115   
4     東京都大田区仲六郷**********               S13001          20170605   

      status_cd age_category era  
0  0-00000000-0           30  30  
1  0-00000000-0           60  60  
2  D-20100325-C           40  40  
3  0-00000000-0           60  60  
4  6-20090929-2           20  20

---
> P-057: 056の抽出結果と性別コード（gender_cd）により、新たに性別×年代の組み合わせを表すカテゴリデータを作成し、10件表示せよ。組み合わせを表すカテゴリの値は任意とする。

---
> P-058: 顧客データ（df_customer）の性別コード（gender_cd）をダミー変数化し、顧客ID（customer_id）とともに10件表示せよ。

---
> P-059: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を平均0、標準偏差1に標準化して顧客ID、売上金額合計とともに10件表示せよ。標準化に使用する標準偏差は、分散の平方根、もしくは不偏分散の平方根のどちらでも良いものとする。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

TIPS:
- query()の引数engineで'python'か'numexpr'かを選択でき、デフォルトはインストールされていればnumexprが、無ければpythonが使われます。さらに、文字列メソッドはengine='python'でないとquery()内で使えません。


---
> P-060: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を最小値0、最大値1に正規化して顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

---
> P-061: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を常用対数化（底10）して顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

---
> P-062: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を自然対数化（底e）して顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

---
> P-063: 商品データ（df_product）の単価（unit_price）と原価（unit_cost）から各商品の利益額を算出し、結果を10件表示せよ。

---
> P-064: 商品データ（df_product）の単価（unit_price）と原価（unit_cost）から、各商品の利益率の全体平均を算出せよ。ただし、単価と原価には欠損が生じていることに注意せよ。

---
> P-065: 商品データ（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。ただし、1円未満は切り捨てること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。

---
> P-066: 商品データ（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を丸めること（四捨五入または偶数への丸めで良い）。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。

---
> P-067: 商品データ（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を切り上げること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。

---
> P-068: 商品データ（df_product）の各商品について、消費税率10％の税込み金額を求めよ。1円未満の端数は切り捨てとし、結果を10件表示せよ。ただし、単価（unit_price）には欠損が生じていることに注意せよ。

---
> P-069: レシート明細データ（df_receipt）と商品データ（df_product）を結合し、顧客毎に全商品の売上金額合計と、カテゴリ大区分コード（category_major_cd）が"07"（瓶詰缶詰）の売上金額合計を計算の上、両者の比率を求めよ。抽出対象はカテゴリ大区分コード"07"（瓶詰缶詰）の売上実績がある顧客のみとし、結果を10件表示せよ。

---
> P-070: レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、顧客データ（df_customer）の会員申込日（application_date）からの経過日数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。

---
> P-071: レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、顧客データ（df_customer）の会員申込日（application_date）からの経過月数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1ヶ月未満は切り捨てること。

---
> P-072: レシート明細データ（df_receipt）の売上日（df_customer）に対し、顧客データ（df_customer）の会員申込日（application_date）からの経過年数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1年未満は切り捨てること。

---
> P-073: レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、顧客データ（df_customer）の会員申込日（application_date）からのエポック秒による経過時間を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。なお、時間情報は保有していないため各日付は0時0分0秒を表すものとする。

---
> P-074: レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、当該週の月曜日からの経過日数を計算し、売上日、直前の月曜日付とともに10件表示せよ（sales_ymdは数値でデータを保持している点に注意）。

---
> P-075: 顧客データ（df_customer）からランダムに1%のデータを抽出し、先頭から10件表示せよ。

---
> P-076: 顧客データ（df_customer）から性別コード（gender_cd）の割合に基づきランダムに10%のデータを層化抽出し、性別コードごとに件数を集計せよ。

---
> P-077: レシート明細データ（df_receipt）の売上金額を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。なお、外れ値は売上金額合計を対数化したうえで平均と標準偏差を計算し、その平均から3σを超えて離れたものとする（自然対数と常用対数のどちらでも可）。結果は10件表示せよ。

---
> P-078: レシート明細データ（df_receipt）の売上金額（amount）を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、ここでは外れ値を第1四分位と第3四分位の差であるIQRを用いて、「第1四分位数-1.5×IQR」を下回るもの、または「第3四分位数+1.5×IQR」を超えるものとする。結果は10件表示せよ。

---
> P-079: 商品データ（df_product）の各項目に対し、欠損数を確認せよ。

---
> P-080: 商品データ（df_product）のいずれかの項目に欠損が発生しているレコードを全て削除した新たな商品データを作成せよ。なお、削除前後の件数を表示させ、079で確認した件数だけ減少していることも確認すること。

---
> P-081: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの平均値で補完した新たな商品データを作成せよ。なお、平均値については1円未満を丸めること（四捨五入または偶数への丸めで良い）。補完実施後、各項目について欠損が生じていないことも確認すること。

---
> P-082: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの中央値で補完した新たな商品データを作成せよ。なお、中央値については1円未満を丸めること（四捨五入または偶数への丸めで良い）。補完実施後、各項目について欠損が生じていないことも確認すること。

---
> P-083: 単価（unit_price）と原価（unit_cost）の欠損値について、各商品のカテゴリ小区分コード（category_small_cd）ごとに算出した中央値で補完した新たな商品データを作成せよ。なお、中央値については1円未満を丸めること（四捨五入または偶数への丸めで良い）。補完実施後、各項目について欠損が生じていないことも確認すること。

---
> P-084: 顧客データ（df_customer）の全顧客に対して全期間の売上金額に占める2019年売上金額の割合を計算し、新たなデータを作成せよ。ただし、売上実績がない場合は0として扱うこと。そして計算した割合が0超のものを抽出し、結果を10件表示せよ。また、作成したデータに欠損が存在しないことを確認せよ。

---
> P-085: 顧客データ（df_customer）の全顧客に対し、郵便番号（postal_cd）を用いてジオコードデータ（df_geocode）を紐付け、新たな顧客データを作成せよ。ただし、1つの郵便番号（postal_cd）に複数の経度（longitude）、緯度（latitude）情報が紐づく場合は、経度（longitude）、緯度（latitude）の平均値を算出して使用すること。また、作成結果を確認するために結果を10件表示せよ。

---
> P-086: 085で作成した緯度経度つき顧客データに対し、会員申込店舗コード（application_store_cd）をキーに店舗データ（df_store）と結合せよ。そして申込み店舗の緯度（latitude）・経度情報（longitude)と顧客住所（address）の緯度・経度を用いて申込み店舗と顧客住所の距離（単位：km）を求め、顧客ID（customer_id）、顧客住所（address）、店舗住所（address）とともに表示せよ。計算式は以下の簡易式で良いものとするが、その他精度の高い方式を利用したライブラリを利用してもかまわない。結果は10件表示せよ。

$$
\mbox{緯度（ラジアン）}：\phi \\
\mbox{経度（ラジアン）}：\lambda \\
\mbox{距離}L = 6371 * \arccos(\sin \phi_1 * \sin \phi_2
+ \cos \phi_1 * \cos \phi_2 * \cos(\lambda_1 − \lambda_2))
$$

---
> P-087: 顧客データ（df_customer）では、異なる店舗での申込みなどにより同一顧客が複数登録されている。名前（customer_name）と郵便番号（postal_cd）が同じ顧客は同一顧客とみなして1顧客1レコードとなるように名寄せした名寄顧客データを作成し、顧客データの件数、名寄顧客データの件数、重複数を算出せよ。ただし、同一顧客に対しては売上金額合計が最も高いものを残し、売上金額合計が同一もしくは売上実績がない顧客については顧客ID（customer_id）の番号が小さいものを残すこととする。

---
> P-088: 087で作成したデータを元に、顧客データに統合名寄IDを付与したデータを作成せよ。ただし、統合名寄IDは以下の仕様で付与するものとする。
>
> - 重複していない顧客：顧客ID（customer_id）を設定
> - 重複している顧客：前設問で抽出したレコードの顧客IDを設定
> 
> 顧客IDのユニーク件数と、統合名寄IDのユニーク件数の差も確認すること。

---
> P-089: 売上実績がある顧客を、予測モデル構築のため学習用データとテスト用データに分割したい。それぞれ8:2の割合でランダムにデータを分割せよ。

---
> P-090: レシート明細データ（df_receipt）は2017年1月1日〜2019年10月31日までのデータを有している。売上金額（amount）を月次で集計し、学習用に12ヶ月、テスト用に6ヶ月の時系列モデル構築用データを3セット作成せよ。

---
> P-091: 顧客データ（df_customer）の各顧客に対し、売上実績がある顧客数と売上実績がない顧客数が1:1となるようにアンダーサンプリングで抽出せよ。

---
> P-092: 顧客データ（df_customer）の性別について、第三正規形へと正規化せよ。

---
> P-093: 商品データ（df_product）では各カテゴリのコード値だけを保有し、カテゴリ名は保有していない。カテゴリデータ（df_category）と組み合わせて非正規化し、カテゴリ名を保有した新たな商品データを作成せよ。

---
> P-094: 093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
>
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|有り|UTF-8|
> 
> ファイル出力先のパスは以下のようにすること
> 
> |出力先|
> |:--:|
> |./data|

---
> P-095: 093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
>
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|有り|CP932|
> 
> ファイル出力先のパスは以下のようにすること。
> 
> |出力先|
> |:--:|
> |./data|

---
> P-096: 093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
>
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|無し|UTF-8|
> 
> ファイル出力先のパスは以下のようにすること。
> 
> |出力先|
> |:--:|
> |./data|

---
> P-097: 094で作成した以下形式のファイルを読み込み、データを3件を表示させて正しく取り込まれていることを確認せよ。
> 
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|有り|UTF-8|

---
> P-098: 096で作成した以下形式のファイルを読み込み、データを3件を表示させて正しく取り込まれていることを確認せよ。
> 
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|ヘッダ無し|UTF-8|

---
> P-099: 093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
>
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |TSV（タブ区切り）|有り|UTF-8|
> 
> ファイル出力先のパスは以下のようにすること
> 
> |出力先|
> |:--:|
> |./data|

---
> P-100: 099で作成した以下形式のファイルを読み込み、データを3件を表示させて正しく取り込まれていることを確認せよ。
> 
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |TSV（タブ区切り）|有り|UTF-8|

# これで１００本終わりです。おつかれさまでした！